In [26]:
!pip install opencv-python

In [28]:
%%cmd
where python

Microsoft Windows [Version 10.0.19045.2251]
(c) Microsoft Corporation. All rights reserved.

(base) C:\Users\suena\PycharmProjects\UCSDBootCamp\capstone\Survey>where python
C:\Users\suena\anaconda3\python.exe
C:\Users\suena\AppData\Local\Microsoft\WindowsApps\python.exe

(base) C:\Users\suena\PycharmProjects\UCSDBootCamp\capstone\Survey>

In [30]:
%%cmd
pip install cmake

Microsoft Windows [Version 10.0.19045.2251]
(c) Microsoft Corporation. All rights reserved.

(base) C:\Users\suena\PycharmProjects\UCSDBootCamp\capstone\Survey>pip install cmake


     --------------------------------------- 32.6/32.6 MB 13.1 MB/s eta 0:00:00



(base) C:\Users\suena\PycharmProjects\UCSDBootCamp\capstone\Survey>

In [31]:
!pip install dlib

  Using cached dlib-19.24.0.tar.gz (3.2 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for dlib: filename=dlib-19.24.0-cp38-cp38-win_amd64.whl size=3053962 sha256=0632311e328a7f05fe56d83835f6f2568d2dec6e6b7eddf7bd54706df1fb8781
  Stored in directory: c:\users\suena\appdata\local\pip\cache\wheels\4c\d8\2d\a83b10e7bf10cd7d8bef36bf4dcd15b0c9ebf98f990bc984dd
Successfully built dlib


In [32]:
!pip install face_recognition

  Using cached face_recognition-1.3.0-py2.py3-none-any.whl (15 kB)


In [1]:
import os
import cv2
import dlib
import numpy as np
import tensorflow as tf
from glob import glob
import matplotlib.pyplot as plt
from PIL import Image
from skimage import io
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Input, Dense, Flatten, Layer, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
%matplotlib inline

In [2]:
import face_recognition
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

In [4]:
def preprocess(file_path):
    # Read in image from file path
    byte_img = tf.io.read_file(file_path)
    # Load in the image
    img = tf.io.decode_jpeg(byte_img)
    # Preprocessing steps - resizing the image to be 150x150
    img = tf.image.resize(img, (150,150))
    # Scale image to be between 0 and 1
    img = img / 255.0
    
    # Return iimage
    return img

In [5]:
def preprocess_twin(input_img, validation_img, label):
    return(preprocess(input_img), preprocess(validation_img), label)

In [6]:
# Build Distance Layer
# Create Siamese L1 distance class
class L1Dist(Layer):
    
    # Init method - inheritance
    def __init__(self, **kwargs):
        super().__init__()
    
    # Similarity calculation
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [7]:
# Reload model
model = tf.keras.models.load_model('siamesemodel_VG16.h5', 
                                   custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

In [8]:
# Capture the faces from webcam
# Establish a connection to the webcam
cam=0
for i in range(4):
    if cv2.VideoCapture(i):
        cam=i
        break
    else:
        print('no camera found')
   
cap = cv2.VideoCapture(cam)

number_of_faces=0
while cap.isOpened():
    ret, frame = cap.read()

    # Create a HOG face detector using the built-in dlib class
    face_detector = dlib.get_frontal_face_detector()

    # Run the HOG face detector on the image data.
    # The result will be the bounding boxes of the faces in our image.
    detected_faces = face_detector(frame, 1)
    number_of_faces = len(detected_faces)
    
    # Loop through each face found in the image
    if number_of_faces == 2:
        for i, face_rect in enumerate(detected_faces):

            # Crop image and save
            crop_image = frame[face_rect.top():face_rect.bottom(), face_rect.left():face_rect.right()]
            cv2.imwrite("face{}.jpg".format(i),crop_image)

        # verify the faces
        face_0 = tf.data.Dataset.list_files('face0.jpg').take(1)
        face_1 = tf.data.Dataset.list_files('face1.jpg').take(1)
        faces = tf.data.Dataset.zip((face_0, face_1, tf.data.Dataset.from_tensor_slices(tf.ones(1)))) 

        faces = faces.map(preprocess_twin)

        faces = faces.take(1)
        faces = faces.batch(1)

        face_1, face_2, y_true = faces.as_numpy_iterator().next()

        prediction = model.predict([face_1, face_2])
        pred = np.round(prediction * 100,2)
        
        # Same person = green box around the faces and "Verified" + percentage on the screen
        if prediction > 0.75:
            for i, face_rect in enumerate(detected_faces):
                cv2.rectangle(frame, (face_rect.left(), face_rect.top()),(face_rect.right(), face_rect.bottom()),(0,255,0),2)
            cv2.putText(frame, f'Verified {pred}%', (200,40), cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,0),2) 

        # Different people = red box around the faces and "Not Verified" + percentage on the screen
        else:
            for i, face_rect in enumerate(detected_faces):
                cv2.rectangle(frame, (face_rect.left(), face_rect.top()),(face_rect.right(), face_rect.bottom()),(0,0,255),2)
            cv2.putText(frame, f'Not Verified {pred}%', (200,40), cv2.FONT_HERSHEY_COMPLEX,0.8,(0,0,255),2) 
   
    # show image back to screen
    cv2.imshow('Image Collection', frame)
    
    # close the window
    if cv2.waitKey(1) &  0XFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 289ms/step


KeyboardInterrupt: 